In [1]:
%matplotlib inline
from __future__ import division, print_function
import os
import hugs
import pandas as pd
import matplotlib.pyplot as plt
import webbrowser
from astropy.table import Table
from toolbox.cats import crossmatch
from toolbox.cosmo import Cosmology
from toolbox.astro import angsep
cosmo = Cosmology()
plt.style.use('jpg')

## Get Data

In [18]:
path = '/Volumes/tigress/hugs-pipe-io/patches-run-20170228-173126/'
butler = hugs.datasets.CatButler(path)
outdir = os.path.join(os.environ.get('HUGS_PIPE_IO'), 'sample-3-27-2017')
cat_fn = os.path.join(outdir, 'cat-3-27-2017.csv')

if os.path.isfile(cat_fn):
    cat = pd.read_csv(cat_fn)
else:
    cat = butler.combine_patch_cats()
    cat_all = cat.copy()
    hugs.datasets.hugged.remove_duplicates(cat)
    cat['g-i'] = cat['MAG_APER_0(g)'] - cat['MAG_APER_0(i)']
    cat['g-r'] = cat['MAG_APER_0(g)'] - cat['MAG_APER_0(r)']
    cat.to_csv(cat_fn, index=False)

## Make Selection

In [39]:
m, b = 0.6724944691862157, -0.3313339303786671
color_line = lambda _x: m*_x + b

cut = cat['mu_aper_0(g)'] > 24.0
cut &= cat['FLUX_RADIUS(i)'] > 3.0/0.168
cut &= np.abs(cat['r_e(i)']-cat['r_e(g)'])<5.0
cut &= np.abs(cat['dr_e(i)']) < 5.0
cut &= np.abs(cat['dmu(i)']) < 1.0
cut &= np.abs(cat['dmu(g)']) < 1.0
cut &= np.abs(cat['dR0']) < 15
cut &= cat['n'] > 0.01
cut &= cat['ell']<0.70

color_cut = (cat['g-r'] > color_line(cat['g-i']))
color_cut &= cat['g-i'] < 1.9
color_cut &= cat['g-i'] > 0.
color_cut &= cat['g-r'] < 1.5
color_cut &= cat['g-r'] > 0.

# no color cut for now
#cut &= color_cut

sample = cat[cut].copy()

print(len(sample), 'candidates in sample')

selection_fn = os.path.join(outdir, 'selection-3-27-2017.csv')
sample.to_csv(selection_fn, index=False)

473 candidates in sample
